In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
import os
import random
from torch.autograd import Variable
import copy
from torch import nn, optim
import torch.optim as optim
import numpy as np
from torch.autograd import Variable
from collections import OrderedDict
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import csv
import time


In [2]:
def fix_seed(seed):
    # random
    random.seed(seed)
    # Numpy
    np.random.seed(seed)
    # Pytorch
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

SEED = 42
fix_seed(SEED)

In [3]:
class Argments():
  def __init__(self):
    self.batch_size = 20
    self.test_batch = 1000
    self.global_epochs = 300
    self.local_epochs = 2
    self.lr = None
    self.momentum = 0.9
    self.weight_decay = 10**-4.0
    self.clip = 20.0
    self.partience = 300
    self.worker_num = 20
    self.sample_num = 20
    self.device = device = torch.device('cuda:0'if torch.cuda.is_available() else'cpu')
    self.criterion = nn.CrossEntropyLoss()

args = Argments()

In [4]:
# Tuned value
lr = 2

In [6]:
lr_list = []
lr_list.append(10**-3.0)
lr_list.append(10**-2.5)
lr_list.append(10**-2.0)
lr_list.append(10**-1.5)
lr_list.append(10**-1.0)
lr_list.append(10**-0.5)
lr_list.append(10**0.0)
lr_list.append(10**0.5)

args.lr = lr_list[lr]

In [9]:
class LocalDataset(torch.utils.data.Dataset):
  def __init__(self,dataset,worker_id):
    self.data = []
    self.target = []
    self.id = worker_id
    for data in dataset.take(len(dataset)):
      self.data.append(torch.tensor([data['pixels'].numpy()]))
      self.target.append(torch.tensor(data['label'].numpy().astype(np.int64)))

  def __getitem__(self, index):
    return self.data[index],self.target[index]

  def __len__(self):
    return len(self.data)

In [10]:
with open('../data/federated_trainset_femnist.pickle', 'rb') as f:
    all_federated_trainset = pickle.load(f)
with open('../data/federated_testset_femnist.pickle', 'rb') as f:
    all_federated_testset = pickle.load(f)
all_worker_num = len(all_federated_trainset)

In [11]:
worker_id_list = random.sample(range(all_worker_num),args.worker_num)
print(worker_id_list)
federated_trainset = []
federated_testset = []
for i in worker_id_list:
    federated_trainset.append(all_federated_trainset[i])
    federated_testset.append(all_federated_testset[i])

[2619, 456, 102, 3037, 1126, 1003, 914, 571, 3016, 419, 2771, 3033, 2233, 356, 2418, 1728, 130, 122, 383, 895]


In [12]:
federated_valset = [None]*args.worker_num
for i in range(args.worker_num):
  n_samples = len(federated_trainset[i])
  if n_samples==1:
    federated_valset[i] = copy.deepcopy(federated_trainset[i])
  else:
    train_size = int(len(federated_trainset[i]) * 0.7) 
    val_size = n_samples - train_size 
    federated_trainset[i],federated_valset[i] = torch.utils.data.random_split(federated_trainset[i], [train_size, val_size])

In [14]:
class CNN1(torch.nn.Module):
    def __init__(self):
        super(CNN1, self).__init__()
        self.conv2d_1 = torch.nn.Conv2d(1, 32, kernel_size=3)
        self.max_pooling = nn.MaxPool2d(2, stride=2)
        self.dropout_1 = nn.Dropout(0.25)
        self.flatten = nn.Flatten()
        self.linear_1 = nn.Linear(5408, 128)
        self.dropout_2 = nn.Dropout(0.5)
        self.linear_2 = nn.Linear(128,62)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.conv2d_1(x)
        x = self.max_pooling(x)
        x = self.dropout_1(x)
        x = self.flatten(x)
        x = self.relu(self.linear_1(x))
        x = self.dropout_2(x)
        x = self.linear_2(x)
        return x

class CNN2(torch.nn.Module):
    def __init__(self):
        super(CNN2, self).__init__()
        self.conv2d_1 = torch.nn.Conv2d(1, 32, kernel_size=3)
        self.max_pooling = nn.MaxPool2d(2, stride=2)
        self.conv2d_2 = torch.nn.Conv2d(32, 64, kernel_size=3)
        self.dropout_1 = nn.Dropout(0.25)
        self.flatten = nn.Flatten()
        self.linear_1 = nn.Linear(9216, 128)
        self.dropout_2 = nn.Dropout(0.5)
        self.linear_2 = nn.Linear(128,62)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.conv2d_1(x)
        x = self.conv2d_2(x)
        x = self.max_pooling(x)
        x = self.dropout_1(x)
        x = self.flatten(x)
        x = self.relu(self.linear_1(x))
        x = self.dropout_2(x)
        x = self.linear_2(x)
        return x

class CNN3(torch.nn.Module):
    def __init__(self):
        super(CNN3, self).__init__()
        self.conv2d_1 = torch.nn.Conv2d(1, 32, kernel_size=3)
        self.max_pooling = nn.MaxPool2d(2, stride=2)
        self.conv2d_2 = torch.nn.Conv2d(32, 64, kernel_size=3)
        self.conv2d_3 = torch.nn.Conv2d(64, 128, kernel_size=3)
        self.dropout_1 = nn.Dropout(0.25)
        self.flatten = nn.Flatten()
        self.linear_1 = nn.Linear(15488, 128)
        self.dropout_2 = nn.Dropout(0.5)
        self.linear_2 = nn.Linear(128,62)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.conv2d_1(x)
        x = self.conv2d_2(x)
        x = self.conv2d_3(x)
        x = self.max_pooling(x)
        x = self.dropout_1(x)
        x = self.flatten(x)
        x = self.relu(self.linear_1(x))
        x = self.dropout_2(x)
        x = self.linear_2(x)
        return x
    
class CNN4(torch.nn.Module):
    def __init__(self):
        super(CNN4, self).__init__()
        self.conv2d_1 = torch.nn.Conv2d(1, 32, kernel_size=3)
        self.max_pooling = nn.MaxPool2d(2, stride=2)
        self.conv2d_2 = torch.nn.Conv2d(32, 64, kernel_size=3)
        self.conv2d_3 = torch.nn.Conv2d(64, 128, kernel_size=3)
        self.conv2d_4 = torch.nn.Conv2d(128, 256, kernel_size=3)
        self.dropout_1 = nn.Dropout(0.25)
        self.flatten = nn.Flatten()
        self.linear_1 = nn.Linear(25600, 128)
        self.dropout_2 = nn.Dropout(0.5)
        self.linear_2 = nn.Linear(128,62)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.conv2d_1(x)
        x = self.conv2d_2(x)
        x = self.conv2d_3(x)
        x = self.conv2d_4(x)
        x = self.max_pooling(x)
        x = self.dropout_1(x)
        x = self.flatten(x)
        x = self.relu(self.linear_1(x))
        x = self.dropout_2(x)
        x = self.linear_2(x)
        return x

In [15]:
class Server():
  def __init__(self):
    self.global_model = CNN2()
 

  def create_worker(self,federated_trainset,federated_valset,federated_testset):
    workers = []
    for i in range(args.worker_num):
      workers.append(Worker(federated_trainset[i],federated_valset[i],federated_testset[i]))
    return workers

  def sample_worker(self,workers):
    sample_worker = []
    sample_num = random.sample(range(args.worker_num),args.sample_num)
    for i in sample_num:
      sample_worker.append(workers[i])
    return sample_worker


  def send_model(self,workers):
    nums = 0
    for worker in workers:
      nums += worker.train_data_num

    for worker in workers:
      worker.aggregation_weight = 1.0*worker.train_data_num/nums
      worker.global_model = copy.deepcopy(self.global_model)


  def aggregate_model(self,workers):   
    new_params = OrderedDict()
    for i,worker in enumerate(workers):
      worker_state = worker.global_model.state_dict()
      for key in worker_state.keys():
        if i==0:
          new_params[key] = worker_state[key]*worker.aggregation_weight
        else:
          new_params[key] += worker_state[key]*worker.aggregation_weight
      del worker.global_model 
    self.global_model.load_state_dict(new_params)

In [16]:
class Worker():
  def __init__(self,trainset,valset,testset):
    self.train_data_num = len(trainset)
    self.test_data_num = len(testset)
    if len(trainset)<=2 :
      trainset = [trainset,copy.deepcopy(trainset),copy.deepcopy(trainset)]
    else:
      split_len = [(len(trainset) + i) // 3 for i in range(3)]
      trainset = torch.utils.data.random_split(trainset,split_len)
    self.trainloader_local = torch.utils.data.DataLoader(trainset[0],batch_size=args.batch_size,shuffle=True,num_workers=2)
    self.trainloader_lambda = torch.utils.data.DataLoader(trainset[1],batch_size=args.batch_size,shuffle=True,num_workers=2)
    self.trainloader_global = torch.utils.data.DataLoader(trainset[2],batch_size=args.batch_size,shuffle=True,num_workers=2)
    self.valloader = torch.utils.data.DataLoader(valset,batch_size=args.test_batch,shuffle=False,num_workers=2)
    self.testloader = torch.utils.data.DataLoader(testset,batch_size=args.test_batch,shuffle=False,num_workers=2)
    self.global_model = None
    self.local_model = CNN2()
    self.mix_model = None
    self.lmd = None
    self.aggregation_weight = None


  def local_train(self):
    loss = float('inf')
    self.mix_model = CNN2()
    model_tmp = CNN2()
    for lmd in np.arange(0.0,1.1,0.1):
      new_params = OrderedDict()
      global_state = self.global_model.state_dict()
      local_state = self.local_model.state_dict()
      for key in global_state.keys():
        new_params[key] = lmd*local_state[key] + (1.0-lmd)*global_state[key]     
      model_tmp.load_state_dict(new_params)
      model_tmp = model_tmp.to(args.device)
      if lmd!=0.0:
        _,_ = train(model_tmp,args.criterion,self.trainloader_local,args.local_epochs)

      _,tmp = test(model_tmp,args.criterion,self.trainloader_lambda)
      if tmp<loss:
        loss = tmp
        self.lmd = lmd
        self.mix_model = copy.deepcopy(model_tmp)

    if self.lmd!=0.0:
      self.mix_model = self.mix_model.to('cpu')
      new_params = OrderedDict()
      mix_state = self.mix_model.state_dict()
      global_state = self.global_model.state_dict()
      for key in global_state.keys():
        new_params[key] = (mix_state[key] - (1.0-self.lmd)*global_state[key])/self.lmd    
      self.local_model.load_state_dict(new_params)
      self.mix_model = self.mix_model.to(args.device)
    
    model_tmp = model_tmp.to('cpu')
    del model_tmp


  def global_train(self):
    model_tmp = copy.deepcopy(self.mix_model)
    model_tmp = model_tmp.to('cpu')
    acc_train,loss_train = train(self.mix_model,args.criterion,self.trainloader_global,args.local_epochs)
    acc_valid,loss_valid = test(self.mix_model,args.criterion,self.valloader)
    self.mix_model = self.mix_model.to('cpu')
    new_params = OrderedDict()
    new_state = self.mix_model.state_dict()
    old_state = model_tmp.state_dict()
    global_state = self.global_model.state_dict()
    for key in new_state.keys():
        new_params[key] = global_state[key]+new_state[key]-old_state[key]  
    self.global_model.load_state_dict(new_params)
    del self.mix_model
    del model_tmp
    return acc_train,loss_train,acc_valid,loss_valid


  def test_step(self):
    self.local_train()
    mix_model = CNN2()
    new_params = OrderedDict()
    global_state = self.global_model.state_dict()
    local_state = self.local_model.state_dict()
    for key in global_state.keys():
      new_params[key] = self.lmd*local_state[key] + (1.0-self.lmd)*global_state[key]     
    mix_model.load_state_dict(new_params)
    mix_model = mix_model.to(args.device)
    acc,loss = test(mix_model,args.criterion,self.testloader)
    return acc,loss

In [17]:
def train(model,criterion,trainloader,epochs):
  optimizer = optim.SGD(model.parameters(),lr=args.lr,momentum=args.momentum,weight_decay=args.weight_decay)
  model.train()
  for epoch in range(epochs):
    running_loss = 0.0
    correct = 0
    count = 0
    for (data,labels) in trainloader:
      data,labels = Variable(data),Variable(labels)
      data,labels = data.to(args.device),labels.to(args.device)
      optimizer.zero_grad()
      outputs = model(data)
      loss = criterion(outputs,labels)
      running_loss += loss.item()
      predicted = torch.argmax(outputs,dim=1)
      correct += (predicted==labels).sum().item()
      count += len(labels)
      loss.backward()
      torch.nn.utils.clip_grad_norm_(model.parameters(), args.clip)
      optimizer.step()

  return 100.0*correct/count,running_loss/len(trainloader)



In [18]:
def test(model,criterion,testloader):
  model.eval()
  running_loss = 0.0
  correct = 0
  count = 0
  for (data,labels) in testloader:
    data,labels = data.to(args.device),labels.to(args.device)
    outputs = model(data)
    running_loss += criterion(outputs,labels).item()
    predicted = torch.argmax(outputs,dim=1)
    correct += (predicted==labels).sum().item()
    count += len(labels)

  accuracy = 100.0*correct/count
  loss = running_loss/len(testloader)


  return accuracy,loss

In [19]:
class Early_Stopping():
  def __init__(self,partience):
    self.step = 0
    self.loss = float('inf')
    self.partience = partience

  def validate(self,loss):
    if self.loss<loss:
      self.step += 1
      if self.step>self.partience:
        return True
    else:
      self.step = 0
      self.loss = loss

    return False

In [20]:
server = Server()
workers = server.create_worker(federated_trainset,federated_valset,federated_testset)
acc_train = []
loss_train = []
acc_valid = []
loss_valid = []

early_stopping = Early_Stopping(args.partience)

start = time.time()

for epoch in range(args.global_epochs):
  sample_worker = server.sample_worker(workers)
  server.send_model(sample_worker)

  acc_train_avg = 0.0
  loss_train_avg = 0.0
  acc_valid_avg = 0.0
  loss_valid_avg = 0.0
  for worker in sample_worker:
    worker.local_train()
    acc_train_tmp,loss_train_tmp,acc_valid_tmp,loss_valid_tmp = worker.global_train()
    acc_train_avg += acc_train_tmp/len(sample_worker)
    loss_train_avg += loss_train_tmp/len(sample_worker)
    acc_valid_avg += acc_valid_tmp/len(sample_worker)
    loss_valid_avg += loss_valid_tmp/len(sample_worker)
  server.aggregate_model(sample_worker)
  '''
  server.model.to(args.device)
  for worker in workers:
    acc_valid_tmp,loss_valid_tmp = test(server.model,args.criterion,worker.valloader)
    acc_valid_avg += acc_valid_tmp/len(workers)
    loss_valid_avg += loss_valid_tmp/len(workers)
  server.model.to('cpu')
  '''
  print('Epoch{}  loss:{}  accuracy:{}'.format(epoch+1,loss_valid_avg,acc_valid_avg))
  acc_train.append(acc_train_avg)
  loss_train.append(loss_train_avg)
  acc_valid.append(acc_valid_avg)
  loss_valid.append(loss_valid_avg)

  if early_stopping.validate(loss_valid_avg):
    print('Early Stop')
    break
    
end = time.time()

Epoch1  loss:4.059211182594299  accuracy:5.549832671338785
Epoch2  loss:4.027985322475433  accuracy:6.0927671324820265
Epoch3  loss:4.007205903530121  accuracy:5.4057809920075135
Epoch4  loss:3.974003493785859  accuracy:6.106197055868956
Epoch5  loss:3.961781740188598  accuracy:5.800641500313399
Epoch6  loss:3.9369140028953558  accuracy:5.83031246491142
Epoch7  loss:3.913912427425385  accuracy:6.123063749845623
Epoch8  loss:3.9010895013809206  accuracy:5.992762462717568
Epoch9  loss:3.8925145268440247  accuracy:5.707121149029074
Epoch10  loss:3.8845389723777766  accuracy:5.476773718681644
Epoch11  loss:3.881003987789154  accuracy:5.9444171863251105
Epoch12  loss:3.859368467330934  accuracy:6.380576265341333
Epoch13  loss:3.8683642983436584  accuracy:5.429677957300168
Epoch14  loss:3.854019463062286  accuracy:6.015556280260558
Epoch15  loss:3.856330084800721  accuracy:6.53281937107281
Epoch16  loss:3.846015954017639  accuracy:6.2701296798116894
Epoch17  loss:3.8370972990989682  accuracy

Epoch136  loss:3.717536747455597  accuracy:8.000074089540865
Epoch137  loss:3.691079139709472  accuracy:9.307444416692823
Epoch138  loss:3.695162439346314  accuracy:7.917212909503564
Epoch139  loss:3.7022746324539186  accuracy:8.866801353355054
Epoch140  loss:3.702975308895111  accuracy:9.302527724047088
Epoch141  loss:3.6879076838493345  accuracy:8.686531604647099
Epoch142  loss:3.688350057601928  accuracy:9.142314547133724
Epoch143  loss:3.6863347768783576  accuracy:9.358799820699033
Epoch144  loss:3.674365317821503  accuracy:8.366600382903258
Epoch145  loss:3.6721840262413035  accuracy:8.9532016900928
Epoch146  loss:3.6749199986457826  accuracy:8.952774211435278
Epoch147  loss:3.6782338500022895  accuracy:9.346722396733602
Epoch148  loss:3.670982682704925  accuracy:8.865408393673667
Epoch149  loss:3.691080284118653  accuracy:8.720411592203433
Epoch150  loss:3.6680558443069455  accuracy:8.87949051011806
Epoch151  loss:3.6860258102416994  accuracy:8.108078681192733
Epoch152  loss:3.68

Epoch270  loss:3.3247191786766055  accuracy:22.916949631678136
Epoch271  loss:3.333003640174865  accuracy:21.645243254642537
Epoch272  loss:3.3244764745235442  accuracy:23.783534064305638
Epoch273  loss:3.3068979859352114  accuracy:22.90360394092968
Epoch274  loss:3.30413174033165  accuracy:22.440409451588145
Epoch275  loss:3.320740073919296  accuracy:22.833053845193593
Epoch276  loss:3.31528360247612  accuracy:21.349671959625258
Epoch277  loss:3.3140181064605714  accuracy:22.22409794413408
Epoch278  loss:3.3040868043899536  accuracy:23.068913497277144
Epoch279  loss:3.292915242910385  accuracy:23.648739626751052
Epoch280  loss:3.290546989440918  accuracy:23.103444954471858
Epoch281  loss:3.309772771596908  accuracy:23.212843989048988
Epoch282  loss:3.30612096786499  accuracy:23.7437597165926
Epoch283  loss:3.302870506048202  accuracy:24.4162324338409
Epoch284  loss:3.278311902284622  accuracy:23.851359802270125
Epoch285  loss:3.2840368866920473  accuracy:23.971004071560312
Epoch286  l

In [23]:
acc_test = []
loss_test = []

nums = 0
for worker in workers:
  nums += worker.test_data_num

start = time.time()

for i,worker in enumerate(workers):
  worker.aggregation_weight = 1.0*worker.test_data_num/nums
  worker.global_model = copy.deepcopy(server.global_model)
  worker.local_train()
  acc_tmp,loss_tmp = test(worker.mix_model,args.criterion,worker.testloader)
  acc_test.append(acc_tmp)
  loss_test.append(loss_tmp)
  print('Worker{} accuracy:{}  loss:{}'.format(i+1,acc_tmp,loss_tmp))
  worker.mix_model = worker.mix_model.to('cpu')
  del worker.mix_model,worker.global_model

end = time.time()

acc_test_avg = sum(acc_test)/len(acc_test)
loss_test_avg = sum(loss_test)/len(loss_test)
print('Test  loss:{}  accuracy:{}'.format(loss_test_avg,acc_test_avg))

Worker1 accuracy:11.764705882352942  loss:3.4357078075408936
Worker2 accuracy:80.0  loss:1.3443061113357544
Worker3 accuracy:47.5  loss:2.2990972995758057
Worker4 accuracy:11.11111111111111  loss:3.8096609115600586
Worker5 accuracy:26.470588235294116  loss:3.3714046478271484
Worker6 accuracy:19.047619047619047  loss:3.4812235832214355
Worker7 accuracy:21.05263157894737  loss:3.5776937007904053
Worker8 accuracy:31.25  loss:3.1965279579162598
Worker9 accuracy:0.0  loss:3.6245222091674805
Worker10 accuracy:46.34146341463415  loss:2.8203229904174805
Worker11 accuracy:16.666666666666668  loss:3.916609048843384
Worker12 accuracy:10.526315789473685  loss:3.3269546031951904
Worker13 accuracy:15.789473684210526  loss:3.2606372833251953
Worker14 accuracy:10.0  loss:3.789907217025757
Worker15 accuracy:26.31578947368421  loss:3.3726754188537598
Worker16 accuracy:17.647058823529413  loss:3.722712278366089
Worker17 accuracy:54.54545454545455  loss:2.1526758670806885
Worker18 accuracy:38.709677419354